In [1]:
import os
import pandas as pd
from tqdm import tqdm
import shutil

In [36]:
test_submission = pd.read_csv('../data/test_submission/bilstm_transformer.csv')

test = pd.read_csv('../data/subtask1_test.csv')

print (test.shape, test_submission.shape)

(872149, 6) (823098, 8)


In [37]:
test_submission.head(10)

,document,sentence,n1,n2,word,pos,doc_sent,tag
0,S0004-06142008000100008-1,0,0,11,presentamos,VERB,S0004-06142008000100008-1_0,OTHER
1,S0004-06142008000100008-1,0,15,19,caso,NOUN,S0004-06142008000100008-1_0,OTHER
2,S0004-06142008000100008-1,0,27,32,mujer,NOUN,S0004-06142008000100008-1_0,OTHER
3,S0004-06142008000100008-1,0,36,38,30,NUM,S0004-06142008000100008-1_0,OTHER
4,S0004-06142008000100008-1,0,39,43,años,NOUN,S0004-06142008000100008-1_0,OTHER
5,S0004-06142008000100008-1,0,45,53,fumadora,ADJ,S0004-06142008000100008-1_0,OTHER
6,S0004-06142008000100008-1,0,57,59,20,NUM,S0004-06142008000100008-1_0,OTHER
7,S0004-06142008000100008-1,0,60,75,cigarrillos/día,NOUN,S0004-06142008000100008-1_0,OTHER
8,S0004-06142008000100008-1,0,88,100,antecedentes,NOUN,S0004-06142008000100008-1_0,OTHER
9,S0004-06142008000100008-1,0,101,111,personales,ADJ,S0004-06142008000100008-1_0,OTHER


In [38]:
test_submission.tag.value_counts()

OTHER            806532
NORMALIZABLES      9735
PROTEINAS          6757
PAD                  74
Name: tag, dtype: int64

In [39]:
train = pd.read_csv('../data/subtask1_train.csv')[['word','tag']]
val = pd.read_csv('../data/subtask1_dev.csv')[['word','tag']]
all_words = pd.concat([train,val],axis=0).drop_duplicates().reset_index(drop=True)

xx = all_words.groupby(['word'])['tag'].nunique().reset_index()
xx.columns = ['word','tag_counts']
xx = xx[xx.tag_counts == 1]
print (xx.shape)

uniq_word_tags = pd.merge(all_words[all_words.tag != 'OTHER'][all_words.tag != 'UNCLEAR'],xx,how='inner')[['word','tag']]
uniq_word_tags.columns = ['word','actual_tag']
uniq_word_tags = uniq_word_tags.drop_duplicates(subset=['word'])

test_submission = pd.merge(test_submission,uniq_word_tags,how='left')

def fix_tag(x):
    if pd.notnull(x.actual_tag) and x.tag != x.actual_tag:
        x['tag'] = x.actual_tag
    return x

test_submission = test_submission.apply(lambda x: fix_tag(x),axis=1)

(24727, 2)


/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [40]:
test_submission.tag.value_counts()

OTHER               789489
NORMALIZABLES        20050
PROTEINAS            13469
PAD                     74
NO_NORMALIZABLES        16
Name: tag, dtype: int64

In [41]:
test_submission = test_submission[test_submission.tag != 'OTHER'][test_submission.tag != 'PAD'][test_submission.tag != 'UNCLEAR']

/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [42]:
test_pred = pd.DataFrame()

In [43]:
i = 0
j = 0

n1 = test_submission['n1'].iloc[i]
n2 = test_submission['n2'].iloc[i]
document = test_submission['document'].iloc[i]
doc_sent = test_submission['doc_sent'].iloc[i]
tag = test_submission['tag'].iloc[i]
word = [test_submission['word'].iloc[i]]

documents = []
n1s = []
n2s = []
tags = []
words = []

while i < test_submission.shape[0]-1:
     
    if test_submission['doc_sent'].iloc[i+1] == doc_sent and \
            test_submission['tag'].iloc[i+1] == tag and test_submission['n1'].iloc[i+1] == n2 + 1 :
            
            n2 = test_submission['n2'].iloc[i+1]
            word += [test_submission['word'].iloc[i+1]]
    
    else:
        
        documents.append(document)
        n1s.append(n1)
        n2s.append(n2)
        tags.append(tag)
        words.append(" ".join(word))
        
        j += 1
        
        n1 = test_submission['n1'].iloc[i+1]
        n2 = test_submission['n2'].iloc[i+1]
        document = test_submission['document'].iloc[i+1]
        doc_sent = test_submission['doc_sent'].iloc[i+1]
        tag = test_submission['tag'].iloc[i+1]
        word = [test_submission['word'].iloc[i+1]]

    i += 1
    
test_pred['document'] = documents
test_pred['n1'] = n1s
test_pred['n2'] = n2s
test_pred['word'] = words
test_pred['tag_pred'] = tags

In [44]:
test.document.nunique()

3751

In [45]:
print (test_pred.shape)
print (test_pred.tag_pred.value_counts())

(32660, 5)
NORMALIZABLES       19453
PROTEINAS           13191
NO_NORMALIZABLES       16
Name: tag_pred, dtype: int64


In [46]:
!rm -r ../data/submissions/bilstm_transformer/

rm: ../data/submissions/bilstm_transformer/: No such file or directory


In [47]:
!mkdir -p ../data/submissions/bilstm_transformer/subtask1/

In [48]:
train_documents = list(pd.read_csv('../data/subtask1_train.csv')['document'].unique())
val_documents = list(pd.read_csv('../data/subtask1_dev.csv')['document'].unique())
print (len(train_documents),len(val_documents))

500 250


In [49]:
for i in tqdm(test.document.unique()):
    if i in train_documents:
        shutil.copy2(os.path.join('../raw_data/train/subtrack1/',i+'.ann'),os.path.join('../data/submissions/bilstm_transformer/subtask1/',i+'.ann'))
    elif i in val_documents:
        shutil.copy2(os.path.join('../raw_data/dev/subtrack1/',i+'.ann'),os.path.join('../data/submissions/bilstm_transformer/subtask1/',i+'.ann'))
    else:
        df = test_pred[test_pred.document == i]
        f = open('../data/submissions/bilstm_transformer/subtask1/{}.ann'.format(i),'a')
        if len(df) > 0:
            for j in range(len(df)):
                f.writelines("T{}\t{} {} {}\t{}\n".format(j+1,df.tag_pred.iloc[j],df.n1.iloc[j],df.n2.iloc[j],df.word.iloc[j]))


100%|██████████| 3751/3751 [00:11<00:00, 324.22it/s]


In [19]:
#lines = "".join(open('../raw_data/background/S0004-06142005000200004-1.txt').readlines())

In [20]:
#lines[3066:3073]

'3 meses'

In [21]:
#train_documents[:3]

['S0004-06142008000100008-1',
 'S1130-01082008001000010-1',
 'S1579-699X2004000400002-1']